***GENERATED CODE FOR heartfailureprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'platelets', 'threshold': 266157.202, 'transformation_label': 'Binarizer'}], 'feature': 'platelets', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '67', 'mean': '277772.99', 'stddev': '97045.17', 'min': '75000.0', 'max': '621000.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'platelets'}, {'feature_label': 'platelets', 'threshold': 266157.202, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('platelets')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run heartfailurepredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	heartfailureclinicalrecordsdataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1613638334717_heart_failure_clinical_records_dataset.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': "/snehal's scenarios/Healthcare models/Heartfaliure/heart_failure_clinical_records_dataset.csv", 'filename': '1613638334717_heart_failure_clinical_records_dataset.csv', 'viewFileName': 'heart_failure_clinical_records_dataset.csv', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run heartfailurepredictionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(heartfailureclinicalrecordsdataset,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "67", "mean": "62.04", "stddev": "10.93", "min": "40.0", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "anaemia", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.42", "stddev": "0.5", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "anaemia"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine_phosphokinase", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "657.99", "stddev": "865.45", "min": "47.0", "max": "5209.0", "missing": "0"}, "updatedLabel": "creatinine_phosphokinase"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "diabetes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.42", "stddev": "0.5", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "diabetes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ejection_fraction", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "39.25", "stddev": "12.27", "min": "15.0", "max": "60.0", "missing": "0"}, "updatedLabel": "ejection_fraction"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "high_blood_pressure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.4", "stddev": "0.49", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "high_blood_pressure"}, {"transformationsData": [{"feature_label": "platelets", "threshold": 266157.202, "transformation_label": "Binarizer"}], "feature": "platelets", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "67", "mean": "277772.99", "stddev": "97045.17", "min": "75000.0", "max": "621000.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "platelets"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "serum_creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "67", "mean": "0.9", "stddev": "0.68", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "serum_creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "serum_sodium", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "135.79", "stddev": "4.17", "min": "121.0", "max": "144.0", "missing": "0"}, "updatedLabel": "serum_sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sex", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.64", "stddev": "0.48", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "sex"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "smoking", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.3", "stddev": "0.46", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "smoking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "128.73", "stddev": "79.78", "min": "10.0", "max": "256.0", "missing": "0"}, "updatedLabel": "time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DEATH_EVENT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "67", "mean": "0.31", "stddev": "0.47", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "DEATH_EVENT"}, {"feature": "platelets_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "67", "mean": "0.42", "stddev": "0.5", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "platelets_binarizer_trans..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run heartfailurepredictionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["age", "anaemia", "creatinine_phosphokinase", "diabetes", "ejection_fraction", "high_blood_pressure", "platelets_binarizer", "serum_creatinine", "serum_sodium", "sex", "smoking", "time"], "DEATH_EVENT")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

